In [1]:
import os
import sys
from dotmap import DotMap

sys.path.append('../../')

from utils.io import recursive_scan2df, load_yaml
from utils.sampling import ConsecutiveSequences

/home/martin/anaconda3/envs/hil/lib/python3.7/site-packages/kornia/augmentation/augmentation.py:1833: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  category=DeprecationWarning,


In [2]:
server = 'personal'
server = DotMap(load_yaml('../../config/servers.yml')[server])

database_name = 'cholec80_splits'
database_df = recursive_scan2df(os.path.join(server.database.location, database_name), postfix='.mp4')
database_df

/home/martin/anaconda3/envs/hil/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,file,folder
0,split_5.mp4,video36
1,split_35.mp4,video36
2,split_7.mp4,video36
3,split_23.mp4,video36
4,split_16.mp4,video36
5,split_36.mp4,video36
6,split_4.mp4,video36
7,split_1.mp4,video36
8,split_2.mp4,video36
9,split_11.mp4,video36


In [3]:
# # individual database post-processing
# # database_df = database_df[database_df.folder != 'videos']
# database_df = database_df[database_df.folder != 'sample_videos']
# # database_df = database_df.iloc[:1]
# database_df

In [4]:
import cv2
import pandas as pd
import numpy as np

from datasets import VideoDataset
from utils.io import load_pickle, save_pickle
from utils.processing import endoscopy

video_paths = [os.path.join(server.database.location, database_name, x.folder, x.file) for _, x in database_df.iterrows()]
frame_rate = 25
duration = 60
clip_length_in_frames = int(frame_rate*duration)
num_workers = 8

# load metadata, if existing
metadata_name = 'cholec80_splits_metadata.pkl'
metadata = None
print('\nLooking for {}'.format(metadata_name))
if os.path.exists(metadata_name):
    metadata = load_pickle(metadata_name)
    print('Found.')
else:
    print('Could not find {}. Generating metadata.'.format(metadata_name))

video_ds = VideoDataset(
            video_paths=video_paths,
            clip_length_in_frames=clip_length_in_frames,
            frames_between_clips=clip_length_in_frames,
            frame_rate=frame_rate,
            precomputed_metadata=metadata,
            num_workers=num_workers,
            permute=False,
            convert_dtype=False
)

# store metadata
if not os.path.exists(metadata_name):
    metadata = video_ds.metadata
    save_pickle(metadata_name, metadata)

# prepare database
database_transforms = pd.DataFrame(
    columns=['database', 'train', 'file', 'pre_transforms', 'aug_transforms', 'auxiliary']
)

# boundary detection parameters
illumination_th = 0.998

# iterate through each video
for batch in video_ds:
    video, augmented_video, frame_rate, video_fps, video_idx, idx = batch
    video_path = database_df.iloc[video_idx].folder
    video_name = database_df.iloc[video_idx].file

    for frame in video:
        frame = frame.numpy()

        # pre-process frame
        mask = endoscopy.bilateralSegmentation(frame, th=0.07)
        center, radius = endoscopy.boundary_detection.ransacBoundaryCircle(mask, fit='numeric', n_iter=1)

        if radius is not None:
            illumination = endoscopy.illuminationLevel(mask, center, radius)

            print('\rIllumination level: {}, threshold: {}'.format(illumination, illumination > 0.998), end='')

            top_left, shape = endoscopy.maxRectangleInCircle(frame.shape, center, radius)
            center, radius = center.astype(int), int(radius)
            top_left, shape = top_left.astype(int), tuple(map(int, shape))
            cv2.circle(frame, (center[1], center[0]), radius, (255, 255, 0), 2)
            cv2.rectangle(frame, (top_left[1], top_left[0]), (top_left[1] + shape[1], top_left[0] + shape[0]), (255, 255, 0), 2)

            cv2.imshow('img', frame[...,::-1])
            cv2.imshow('mask', mask)
            cv2.waitKey(1)

            if illumination > illumination_th:
                row = {
                    'database': 'cholec80_splits',
                    'train': True,
                    'file': {
                        'name': video_name,
                        'path': video_path
                    },
                    'pre_transforms': [
                        {
                            'module': 'utils.transforms',
                            'type': 'Crop',
                            'kwargs': {
                                'top_left_corner': [top_left[0], top_left[1]],
                                'shape': [shape[0], shape[1]],
                                'order': 'chw'
                            }
                        },
                        {
                            'module': 'torchvision.transforms',
                            'type': 'Resize',
                            'kwargs': {
                                'size': [240, 320]
                            }
                        }
                    ],
                    'aug_transforms': [],
                    'auxiliary': {}
                }

                database_transforms = database_transforms.append(row, ignore_index=True)
                break

# database_transforms.to_pickle('cholec80_splits_transforms.pkl')
cv2.destroyAllWindows()

/home/martin/anaconda3/envs/hil/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/martin/anaconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:116: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "

Looking for cholec80_splits_metadata.pkl
Found.
../../utils/processing/endoscopy/endoscopy/processing.py:187: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use 

In [4]:
import os
import pandas as pd
from dotmap import DotMap

from utils.io import load_yaml
from utils.io import load_pickle, save_pickle
from utils.transforms import anyDictListToCompose
from lightning_data_modules import VideoDataModule


meta_df_name = 'cholec80_splits_transforms.pkl'
meta_df = pd.read_pickle(meta_df_name)
meta_df.aug_transforms = None  # no transforms for evaluation

# load cholec80 splits
prefix = server.database.location
clip_length_in_frames = 2
frames_between_clips = 1
frame_rate = 5
train_split = 0.5
batch_size = 1
num_workers = 8
random_state = 42

dm = VideoDataModule(
    meta_df,
    prefix=prefix,
    clip_length_in_frames=clip_length_in_frames,
    frames_between_clips=frames_between_clips   ,
    frame_rate=frame_rate,
    train_split=train_split,
    batch_size=batch_size,
    num_workers=num_workers,
    random_state=random_state
)

# train_md, _, _ = dm.setup()

/home/martin/anaconda3/envs/hil/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: 'VideoDataModule' object has no attribute '_has_setup_fit'

In [17]:
import cv2
from kornia import tensor_to_image

train_dl = dm.train_dataloader()

for batch in train_dl:
    vid, aug_vid, frame_rate, video_fps, video_idx, idx = batch

    for frame in vid:
        cv2.imshow('frame', tensor_to_image(frame))
        cv2.waitKey(1)

cv2.destroyAllWindows()


/home/martin/anaconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/home/martin/anaconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/home/martin/anaconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be removed in a follow-up version. Please use pts_unit 'sec'.
  "The pts_unit 'pts' gives wrong results and will be removed in a "
/home/martin/anaconda3/envs/hil/lib/python3.7/site-packages/torchvision/io/video.py:105: UserWarning: The pts_unit 'pts' gives wrong results and will be remo

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/martin/anaconda3/envs/hil/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 185, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/martin/anaconda3/envs/hil/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/martin/anaconda3/envs/hil/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "../../datasets/video_dataset.py", line 77, in __getitem__
    video = self._pre_transforms[video_idx](video)
  File "/home/martin/anaconda3/envs/hil/lib/python3.7/site-packages/torchvision/transforms/transforms.py", line 61, in __call__
    img = t(img)
  File "/home/martin/anaconda3/envs/hil/lib/python3.7/site-packages/torchvision/transforms/tran